In [ ]:
meta_path = "D:/models/uno/best_model.pt"


isee_reference = "F:/isee_dataset/11158/label"
congrid_input = "D:/data/inputs/11158/congrid"

energy_isee_path = "D:/data/metric/series/11158/energy_isee.csv"
energy_pinocongrid_path = "D:/data/metric/series/11158/energy_pinocongrid.csv"

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

from rtmag.process.paper.load import MyModel
from rtmag.process.paper.utils import get_label_xyz, get_pot, input_to_model_input, energy, get_model_input

In [ ]:
mm = MyModel(meta_path)

In [ ]:
isee_reference = Path(isee_reference)
isee_reference_files = sorted(isee_reference.glob("*.npz"))

congrid_input = Path(congrid_input)
congrid_input_files = sorted(congrid_input.glob("*.npz"))

In [ ]:
energy_isee_path = Path(energy_isee_path)
energy_isee_path.parent.mkdir(parents=True, exist_ok=True)

energy_pinocongrid_path = Path(energy_pinocongrid_path)
energy_pinocongrid_path.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
if not energy_isee_path.exists():
    results = []
    for isee_ref_file in tqdm(isee_reference_files):
        res = {}
        timestamp = isee_ref_file.stem[12:]
        B, x, y, z = get_label_xyz(isee_ref_file)
        Bp = get_pot(isee_ref_file)
        dx = x[1] - x[0]
        dy = y[1] - y[0]
        dz = z[1] - z[0]
        e = energy(B)
        ep = energy(Bp)
        res["obstime"] = timestamp
        res["energy"] = e
        res["energy_pot"] = ep
        res["dx"] = dx
        res["dy"] = dy
        res["dz"] = dz

        results.append(res)
        df = pd.DataFrame.from_dict(results)
        df.to_csv(energy_isee_path, index=False)

In [ ]:
if not energy_pinocongrid_path.exists():
    results = []
    for congrid_path in tqdm(congrid_input_files):
        res = {}
        timestamp = congrid_path.stem[14:-4]
        model_input, x, y = get_model_input(congrid_path, ret=False)
        dx = x[1] - x[0]
        dy = y[1] - y[0]
        b = mm.get_pred_from_numpy(model_input, ret=False)
        e = energy(b)
        res["obstime"] = timestamp
        res["energy"] = e
        res["dx"] = dx
        res["dy"] = dy

        results.append(res)
        df = pd.DataFrame.from_dict(results)
        df.to_csv(energy_pinocongrid_path, index=False)